In [2]:
pip install intel_extension_for_pytorch

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import time
import intel_extension_for_pytorch as ipex

class NeuralNetwork(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(NeuralNetwork, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, output_size)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)
        return x

def train_model_neural_network(hidden_size, learning_rate, optimize=False):
    # Load data
    X_train, y_train = pd.read_csv('x_train.csv'), pd.read_csv('y_train.csv').values.ravel()
    X_train = torch.from_numpy(X_train.values).float()
    y_train = torch.from_numpy(y_train).long()

    # Define the model
    input_size = X_train.shape[1]
    output_size = len(torch.unique(y_train))
    model = NeuralNetwork(input_size, hidden_size, output_size)

    # Define the loss function and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=learning_rate)

    if optimize:
        model = ipex.optimize(model)
        # Enable Intel oneAPI PyTorch
#         model = model.to('oneapi')
#         model = model.to(torch.float32)
#         model = model.to(torch.intel.cpu)
        

    # Start training
    start_time = time.time()
    for epoch in range(100):
        optimizer.zero_grad()
        outputs = model(X_train)
        loss = criterion(outputs, y_train)
        loss.backward()
        optimizer.step()
    end_time = time.time()

    # Print training time
    print(f'Training Time: {end_time - start_time} seconds\n')

    # Return the trained model
    return model

# Train the model without optimization
trained_model1 = train_model_neural_network(hidden_size=150, learning_rate=0.001, optimize=False)

# Train the model with optimization
trained_model2 = train_model_neural_network(hidden_size=150, learning_rate=0.001, optimize=True)